## TEST Code for running and evaluating Steadiness / Cohesiveness and other metrics

#### Import packages & set dataset path

In [1]:
from helper import *
import sys
import importlib
import numpy as np

## Import Metrics
sys.path.append("/home/hj/vis2021_snc/metrics4mdp/metrics4mdp")
from provider import MDPMetricProvider

## Import SNC
sys.path.append("/home/hj/vis2021_snc/steadiness_cohesiveness")
import snc as sc





PATH_TO_DATASET = "../../dataset-generator/dataset/"

#### Pilot Dataset Load

In [2]:
mnist_umap_path = make_path(PATH_TO_DATASET, "mnist", "umap", 2, [0.5, 10])
emb_data, raw_data, label_data = get_data(mnist_umap_path)

raw_data = np.array(raw_data)
emb_data = np.array(emb_data)
label_data = np.array(label_data)


print(emb_data[:4])
print(len(emb_data))

[[11.29208279  5.92692423]
 [11.54898739  9.0832119 ]
 [11.57514477  0.06624129]
 [10.87285995  1.90799904]]
5000


#### Pilot Local Metrics Usage

In [3]:
# mlist = ["Sammon", "DTM", "DTM_KL1", "Trustworthiness", "Continuity", "MRRE_XZ", "MRRE_ZX"]
# metric_provider = MDPMetricProvider(raw_data, emb_data, mlist, 7) 
# result = metric_provider.run()
# print(result)

In [4]:
## reload SNC for the testing
importlib.reload(sc)
SNC = sc.SNC

parameter = {
    "k": 20
}
snc = SNC(raw_data, emb_data, iteration=300, cluster_parameter = parameter)
snc.fit()

[[2400  247 1846 ... 2733 3467 2749]
 [ 102 1929 1693 ... 1338 2055 1209]
 [2245 1165 2030 ... 1925  580   12]
 ...
 [1819 2881 4425 ... 4362 2869 4300]
 [ 404 2798  204 ... 4902 4353 3459]
 [3068 4630 4800 ... 1945 4079 3620]]
20
5000
[[2400  247 4437 ... 2655 1131  892]
 [1660 1693  102 ... 1114 2329 1151]
 [3848  570 4642 ... 2487 3114 1639]
 ...
 [2869 2016 1819 ... 3096 3160 1726]
 [4671 4553 4337 ... 4154 2665 1869]
 [3068 4066 1778 ... 4135 3071 1465]]
20
5000
